### NOTEBOOK PARA CALCULAR LA MÉTRICA
GENERAR CÓDIGO QUE CALCULA LA MÉTRICA QUE SE USARÁ PARA EVALUAR LOS DATOS

**MÉTRICA WRMSSE usada en competencia oficial** (codigo usado como referencia): 
- https://github.com/pmrgn/m5-wrmsse
- https://www.kaggle.com/code/mubashir1/one-tutorial-to-understand-all-m5-forecasting

In [1]:
import os
# set path root of repo
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('/')[:-1]
root_path = '/'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  /Users/joseortega/Documents/GitHub/forecasting-m5-dataset


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import root_mean_squared_error

In [3]:
import warnings
warnings.filterwarnings("ignore")

### 1. Read raw files

In [25]:
folder_data_raw_dtype = 'data/data_input_dtype/'

df_calender = pd.read_pickle(folder_data_raw_dtype + 'calendar.pkl')
df_prices = pd.read_pickle(folder_data_raw_dtype + 'sell_prices.pkl')
df_sales = pd.read_pickle(folder_data_raw_dtype + 'sales_train_evaluation.pkl')
df_sample_output = pd.read_pickle(folder_data_raw_dtype + 'sample_submission.pkl')

In [5]:
df_sales.head(3) # df con las ventas

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0


In [6]:
df_sample_output

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Generar dataframe ejemplo para valores forecast (template del formato y tamaño necesario)

In [7]:
df_sample_output.shape[0] / 2

30490.0

In [8]:
# head data output formato
example_output_df = df_sample_output[30490:]
example_output_df.head(3)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30490,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30491,HOBBIES_1_002_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30492,HOBBIES_1_003_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# tamaño data output formato
example_output_df.shape

(30490, 29)

**SON 30490 SERIES DONDE LA PRIMERA COLUMNA ES EL ID DE LA SERIE Y LAS RESTANTES 28 COLUMNAS CORRESPONDES A LAS FECHAS DE FCST**

### 3. Generar código para métrica WRMSSE
code base: 
- https://github.com/pmrgn/m5-wrmsse
- https://www.kaggle.com/code/mubashir1/one-tutorial-to-understand-all-m5-forecasting

ESTÁ MEDIA ENREDADA LA MÉTRICA. ASI QUE PARA AGILIZAR LAS PRUEBAS SE CREARAN 2 MÉTRICAS PONDERADAS MÁS SIMPLES **WRMSE** (calcular RMSE para cada serie y ponderar por volumen) y **WAPE** (sum error / sum true)

In [10]:
def _rmsse(test, forecast, train_mse):
    """
    Calculate the Root Mean Squared Scaled Error of the
    aggregated forecast dataframe, which is to be of shape
    (42840,28). 
    """
    forecast_mse = np.mean((test - forecast)**2, axis=1)
    return np.sqrt(forecast_mse / train_mse)

### 4. Calcular WRMSE
- Weigted RMSE
- Calcular RMSE para cada serie y luego ponderar por volumen de cada serie. Elección de métrica más fácil y simple para poder probar

#### 4.1 Generar dataframes true y pred que tengan el formato aceptado
- La primera columna sea "id", la cual corresponde al id de la serie
- Todos los valores (tanto reales como predichos) tiene que estar en las siguientes columnas
- Así se tiene una matriz de la forma (S,H) donde S: total de las series // H: total horizonte predicción

In [24]:
df_sales.iloc[:,-28:]

,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,0.0,0.0,0.0,2.0,0.0,3.0,5.0,0.0,0.0,1.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0
3,0.0,0.0,1.0,2.0,4.0,1.0,6.0,4.0,0.0,0.0,...,1.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,6.0
4,1.0,0.0,2.0,3.0,1.0,0.0,3.0,2.0,3.0,1.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,2.0,0.0,...,1.0,0.0,3.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
30486,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
30487,0.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,2.0,0.0,1.0,0.0,1.0,0.0,2.0
30488,1.0,3.0,0.0,1.0,2.0,1.0,0.0,2.0,1.0,1.0,...,1.0,1.0,1.0,4.0,6.0,0.0,1.0,1.0,1.0,0.0


In [11]:
###### generar dataframe true ######
# del df con las ventas. filtrar las últimas 28 observaciones datos para test. TRUE VALUES
data_true = df_sales.iloc[:,-28:]
data_true = pd.concat([df_sales['id'], 
                       data_true
                      ], axis=1)

In [12]:
###### generar dataframe pred ######
# data predicha usada de ejemplo. dejar en el tipo de dato correcto
example_output_df['id'] = example_output_df['id'].astype('category')

# hacer que df true y df pred tengan los mismo nombre de columnas (AQUI ESTOY SUPONIENDO QUE EL ORDEN ESTÁ CORRECTO)
data_true.columns = example_output_df.columns

In [13]:
# print format data true
data_true.head(3)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.0,0.0,0.0,2.0,0.0,3.0,5.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_evaluation,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_evaluation,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0


In [14]:
# print format data pred
example_output_df.head(3)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30490,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30491,HOBBIES_1_002_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30492,HOBBIES_1_003_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### 4.2 Calcular WRMSE. con for. for por cada serie y calcular RMSE de forma individual y luego ponderar por eso

In [15]:
def calculate_metrics_wrmse_v1(df_true, df_pred):
    """
    Calcular weighted root mean square error. Calcular RMSE para cada serie y ponderarlo por peso. Donde peso se calcula 
    a partir del volumen de los reales de cada serie
    Métrica se calcula haciendo un FOR por serie
    
    Args
        df_true (DataFrame): DataFrame de la forma (S,H) donde S: total de las series // H: total horizonte predicción
        df_pred (DataFrame): DataFrame de la forma (S,H) donde S: total de las series // H: total horizonte predicción
    """

    # generar df con todas las series
    df_metrics = df_true[['id']]
    
    # calcular métrica RMSE para cada serie individual y gurdar en df metrics
    list_all_series = df_metrics['id'].tolist()
    for n_serie in list_all_series:
        # print(n_serie)
    
        # generar serie auxiliar
        aux_true_serie = df_true[df_true['id'] == n_serie]
        aux_pred_serie = df_pred[df_pred['id'] == n_serie]
        
        aux_true_serie = aux_true_serie.drop(columns = 'id').T
        aux_pred_serie = aux_pred_serie.drop(columns = 'id').T
        
        # calcular métrica
        rmse_aux = root_mean_squared_error(y_true = aux_true_serie,
                                           y_pred = aux_pred_serie)
        
        # guardar en df_metrics. columna "rmse"
        df_metrics.loc[df_metrics['id'] == n_serie, 'rmse'] = rmse_aux
    
    
    # calcular peso CADA SERIE en base a todos los datos true (peso = volumen_serie_i / volumen_total_series). 
    df_true_sum_each_serie = df_true.set_index(['id']).sum(axis = 1).to_frame(name='volumen_real_serie').reset_index() 
    df_true_sum_each_serie['weight'] = df_true_sum_each_serie['volumen_real_serie'] / df_true_sum_each_serie['volumen_real_serie'].sum()
    df_true_sum_each_serie.drop(columns = 'volumen_real_serie', inplace = True)
    df_metrics = df_metrics.merge(df_true_sum_each_serie, on='id')
    
    
    # calcular métrica final ponderando rmse * weight (para cada serie)
    df_metrics['weight_rmse'] = df_metrics['rmse'] * df_metrics['weight']
    output_wrmse = df_metrics['weight_rmse'].sum()
    
    return output_wrmse

In [16]:
%%time
calculate_metrics_wrmse_v1(df_pred = example_output_df,
                           df_true = data_true
                          )

CPU times: user 23.4 s, sys: 200 ms, total: 23.6 s
Wall time: 23.6 s


9.067654982685724

#### 4.3 Calcular WRMSE. De forma MATRICIAL. MELT de la data Y luego usar GROUP por serie para EVITAR USAR UN FOR

In [17]:
def calculate_metrics_wrmse_v2(df_true, df_pred):
    """
    Calcular weighted root mean square error. Calcular RMSE para cada serie y ponderarlo por peso. Donde peso se calcula 
    a partir del volumen de los reales de cada serie
    Métrica se calcula haciendo GROUPBY POR SERIE y a la agrupación calcular el RMSE. EVITAR USAR UN FOR.
    
    Args
        df_true (DataFrame): DataFrame de la forma (S,H) donde S: total de las series // H: total horizonte predicción
        df_pred (DataFrame): DataFrame de la forma (S,H) donde S: total de las series // H: total horizonte predicción
    """

    # melt data. dejar en formato "id", "ds", "value"
    df_pred_melt = pd.melt(df_pred, id_vars=['id'], var_name='ds', value_name='pred')
    df_true_melt = pd.melt(df_true, id_vars=['id'], var_name='ds', value_name='true')

    
    # concatenar ambos df true y df pred. # obtener dataframe de la forma "id", "serie", "true", "pred"
    df_true_pred = pd.merge(df_pred_melt, df_true_melt, on = ['id', 'ds'])

    
    # CALCULAR RMSE PARA CADA SERIE
    # definir función para calcular métrica RMSE tras agrupar
    def calculate_rmse(group):
        """
        calcular métrica RMSE individual para cada serie. groupby.
        Es una función se aplica en un .apply(lambda) para la data agrupada por groupby
        """
        y_true = group['true'].values
        y_pred = group['pred'].values
        return root_mean_squared_error(y_true, y_pred)

    # agrupar la data por id. a la agrupación aplicar función que calcula el RMSE de cada serie
    df_metrics = df_true_pred.groupby('id').apply(lambda group: calculate_rmse(group)).reset_index(name='rmse')
    
    
    # calcular peso CADA SERIE en base a todos los datos true (peso = volumen_serie_i / volumen_total_series). 
    df_true_sum_each_serie = df_true.set_index(['id']).sum(axis = 1).to_frame(name='volumen_real_serie').reset_index() 
    df_true_sum_each_serie['weight'] = df_true_sum_each_serie['volumen_real_serie'] / df_true_sum_each_serie['volumen_real_serie'].sum()
    df_true_sum_each_serie.drop(columns = 'volumen_real_serie', inplace = True)
    df_metrics = df_metrics.merge(df_true_sum_each_serie, on='id')
    
    
    # calcular métrica final ponderando rmse * weight (para cada serie)
    df_metrics['weight_rmse'] = df_metrics['rmse'] * df_metrics['weight']
    output_wrmse = df_metrics['weight_rmse'].sum()
    
    return output_wrmse

In [18]:
%%time
calculate_metrics_wrmse_v2(df_pred = example_output_df,
                           df_true = data_true
                          )

CPU times: user 3.24 s, sys: 178 ms, total: 3.42 s
Wall time: 3.34 s


9.067654982685724

In [26]:
# SI TODOS LOS FORECAST FUERAN 0. LA MÉTRICA ES 9.06
# ESTA ES LA PEOR MÉTRICA POSIBLE (a menos que fcst sea negativo y no se corriga para llevarlo a cero)

In [20]:
data_true

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.0,0.0,0.0,2.0,0.0,3.0,5.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_evaluation,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_evaluation,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0
3,HOBBIES_1_004_CA_1_evaluation,0.0,0.0,1.0,2.0,4.0,1.0,6.0,4.0,0.0,...,1.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,6.0
4,HOBBIES_1_005_CA_1_evaluation,1.0,0.0,2.0,3.0,1.0,0.0,3.0,2.0,3.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,2.0,...,1.0,0.0,3.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
30486,FOODS_3_824_WI_3_evaluation,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
30487,FOODS_3_825_WI_3_evaluation,0.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,0.0,...,0.0,0.0,1.0,2.0,0.0,1.0,0.0,1.0,0.0,2.0
30488,FOODS_3_826_WI_3_evaluation,1.0,3.0,0.0,1.0,2.0,1.0,0.0,2.0,1.0,...,1.0,1.0,1.0,4.0,6.0,0.0,1.0,1.0,1.0,0.0


In [19]:
example_output_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30490,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30491,HOBBIES_1_002_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30492,HOBBIES_1_003_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30493,HOBBIES_1_004_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30494,HOBBIES_1_005_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
